In [118]:
import os
#from google.colab import drive
#drive.mount('/content/drive')
#os.chdir('/content/drive/MyDrive/Github/dus_mm/')
import numpy as np
import pandas as pd
import utils
import glob

from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [184]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ="float")], ignore_index = 1)
X = X.loc[:100*4+3]
y = y.loc[:100]
X = utils.init_in(X)
y = utils.decomposition(y)

In [4]:
class Layer():
    def __init__(self, neurons: int = 256, activation: str = 'linear'):
        self.neurons = neurons
        self.activation = activation

In [171]:
class NN_model():
  
    def __init__(self, input_dimension: int):
        self.sequential = [Layer(input_dimension)]
        self.theta = np.array([])
        self.loss = 'mean_squared_error'
        self.optimizer = 'grad_descent' #'adam'
        self.num_iterations = 200
        self.beta_1 = 0.9
        self.beta_2 = 0.9
        self.epsilon = 0.01
        self.step_size = 0.1

    def add(self, layer: Layer):
        self.sequential.append(layer)
        layer_in = self.sequential[-2]
        layer_out = self.sequential[-1]
        gamma = np.sqrt(6)/np.sqrt(layer_in.neurons + layer_out.neurons)
        init_theta = np.random.rand(layer_out.neurons,layer_in.neurons + 1) * 2 * gamma - gamma
        self.theta = np.hstack([self.theta, init_theta.ravel()])

    def activation(self, layer: Layer, x, diff: bool = False):
        if (layer.activation == 'relu'):
            if (diff == True):
                x[x >= 0] = 1
                x[x < 0] = 0
            else:
                x[x < 0] = 0
        elif (layer.activation == 'linear'):
            if (diff == True):
                x = 1
            else:
                x = x
        elif (layer.activation == 'sigmoid'):
            if (diff == True):
                x = self.activation(layer, x) * (1 - self.activation(layer, x))
            else:
                x = 1/(1 + np.exp(-x))
        return x

    def feedforward(self, x):
        a = x
        a_out = np.array(a.ravel())
        slice_theta1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[i - 1]
            layer2 = self.sequential[i]
            slice_theta2 = slice_theta1 + (layer1.neurons + 1) * layer2.neurons
            
            theta1 = self.theta[slice_theta1:slice_theta2].reshape(layer2.neurons, layer1.neurons + 1)
            z = np.dot(np.concatenate((np.ones([a.shape[0],1]), a), axis = 1), theta1.T)
            a = self.activation(layer2, z)
            a_out = np.hstack([a.ravel(), a_out])
            slice_theta1 = slice_theta2
        return a_out

    def backprop(self, a_out, y): 
        grad_out = np.array([])
        slice_theta1 = 0
        slice_a1 = 0
        for i in range(1, len(self.sequential)):
            layer1 = self.sequential[-i]
            layer2 = self.sequential[-(i + 1)]
            slice_theta2 = slice_theta1 + layer1.neurons * (layer2.neurons + 1)
            slice_a2 = slice_a1 + layer1.neurons * y.shape[0]
            slice_a3 = slice_a2 + layer2.neurons * y.shape[0]           
            a1 = a_out[slice_a1:slice_a2].reshape((y.shape[0], layer1.neurons))
            a2 = a_out[slice_a2:slice_a3].reshape((y.shape[0], layer2.neurons))
            a2 = np.concatenate((np.ones([a2.shape[0], 1]), a2), axis = 1)
            theta2 = np.flip(self.theta)[slice_theta1:slice_theta2].reshape(layer1.neurons, layer2.neurons + 1)
            theta2 = np.flip(theta2)
            if (i == 1):
                delta = self.compute_loss(a1, y, diff = True) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)
            else:
                delta = np.dot(delta, theta1[:,1:]) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)  
            grad = (1/y.shape[0]) * np.dot(delta.T, a2)
            grad_out = np.hstack([grad.ravel(), grad_out])
            
            theta1 = theta2  
            slice_theta1 = slice_theta2
            slice_a1 = slice_a2
        return grad_out

    def compute_loss(self, a_end, y, diff: bool = False):
        if (self.loss == 'mean_squared_error'):
            if (diff == True):
                loss = (a_end - y)
            else:
                loss = np.sum(np.power((a_end - y), 2))/(2 * y.size)
        elif (self.loss == 'loglos'):
            if (diff == True):
                loss = (a_end - y)/((1 - a_end) * a_end)
            else:
                loss = - np.sum(y * np.log(a_end) + (1 - y) * np.log(1 - a_end))/y.shape[0]
        return loss
        
    def optimize(self, x, y):
        x = np.array(x)
        y = np.array(y)
        if (self.optimizer == 'adam'):
            m = np.zeros(self.theta.shape)
            v = np.zeros(self.theta.shape)
            for t in range(1,self.num_iterations + 1):
                a_out = self.feedforward(x)
                a_end = a_out[:(self.sequential[-1].neurons * y.shape[0])]
                loss = self.compute_loss(a_end, y.ravel())
                grad = self.backprop(a_out, y)
                m = self.beta_1 * m + (1 - self.beta_1) * grad
                v = self.beta_2 * v + (1 - self.beta_2) * np.power(grad, 2)
                m_hat = m / (1 - np.power(self.beta_1, t))
                v_hat = v / (1 - np.power(self.beta_2, t))
                self.theta = self.theta - self.step_size * m_hat / (np.sqrt(v_hat) + self.epsilon)
                print(loss)
        if (self.optimizer == 'grad_descent'):
            for t in range(1,self.num_iterations + 1):
                a_out = self.feedforward(x)
                a_end = a_out[:(self.sequential[-1].neurons * y.shape[0])]
                loss = self.compute_loss(a_end,y.ravel())
                grad = self.backprop(a_out, y)
                self.theta = self.theta - self.step_size * grad
                print(loss)
                    
    def __repr__(self):
        return str(len(self.sequential))


In [185]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(y.shape[1], 'linear'))

In [186]:
dus_mm.optimize(X, y)

48.883097212885716
37.74385699825497
530458.4916068681
37342067006.47152
5.0713678283669444e+67
6.814918977392284e+279
inf
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

<ipython-input-171-37148a4a3c0c>:75: RuntimeWarning: invalid value encountered in multiply
  delta = np.dot(delta, theta1[:,1:]) * self.activation(layer1, np.dot(a2, theta2.T), diff = True)



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [187]:
import scipy.io
mat = scipy.io.loadmat('mnist.mat')
X = mat['X']
y = mat['y']
y[y == 10] = 0
y0 = np.zeros((y.size, 10))
for i in range(y.shape[0]):
    y0[i,y[i]] = 1

In [188]:
dus_mm = NN_model(X.shape[1])
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(64, 'relu'))
dus_mm.add(Layer(y0.shape[1], 'relu'))

In [ ]:
dus_mm.optimize(X, y0)

0.0402916698216308
0.03987291729350936
0.03944715236336585
0.03902251697403065
0.03860815889945111
0.03820851080800904
0.03782062865058678
0.03745091168000823
0.0371016096928583
0.036771617735910075
0.03645542260028445
0.03615220377016762
0.03585968072816807
0.03557729027150622
0.035305153509449495
0.035043070538496306
0.034789210217107486
0.03454240580294097
0.03430338788449475
0.034071377078682775
0.03384668972151334
0.03362854404311345
0.033415417074011036
0.03320888915367642
0.033008938657654974
0.03281454134706558
0.03262519759408449
0.032441808044796336
0.032263667948632176
0.03209023487406271
0.03192138397212469
0.031756943834696114
0.03159627739628141
0.03143935237998811
0.031287183154009995
0.031139061142348018
0.030994679917121053
0.030854401752806463
0.030717872453396168
0.030584795633864893
0.030455027170317592
0.030328437785486544
0.03020464000681457
0.030083917248888037
0.029966137685421657
0.029850857850001394
0.029737944967884504
0.029627801174660708
0.02952014928587425

In [192]:
dus_mm.feedforward(X[600:601,:])[:10]

array([0.        , 0.        , 0.        , 0.10041575, 0.        ,
       0.        , 0.        , 0.        , 0.1994442 , 0.        ])

In [183]:
y0[600]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
 a = [1,2,3]

In [134]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
np.flip(a)

array([3, 2, 1])